In [14]:
import pandas as pd
import numpy as np
from datetime import datetime
from tqdm import tqdm_notebook
from IPython.display import clear_output

### Split by CustomerInterest

In [15]:
df_trade = pd.read_csv('Trade_by_week.csv')
df_trade.drop(['TradeDateKey', 'NotionalEUR', 'Price', 'TradeStatus', 'day_of_week', 'day_of_month', 'date'], axis=1, inplace=True)

df_ones = df_trade[df_trade['CustomerInterest'] == 1]
df_zeros = df_trade[df_trade['CustomerInterest'] == 0]

print(df_ones.shape, df_zeros.shape)

(2211929, 6) (4550092, 6)


### Split by BuySell

In [16]:
df_ones_buy = df_ones[df_ones['BuySell'] == 'Buy']
df_ones_sell = df_ones[df_ones['BuySell'] == 'Sell']

print(df_ones_buy.shape, df_ones_sell.shape)

(1195889, 6) (1016040, 6)


### Create opposite directions

In [17]:
df_zeros_sell = df_ones_buy
df_zeros_sell['BuySell'] = 'Sell'
df_zeros_sell['CustomerInterest'] = 0

df_zeros_buy = df_ones_sell
df_zeros_buy['BuySell'] = 'Buy'
df_zeros_buy['CustomerInterest'] = 0

/home/nando/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/home/nando/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/home/nando/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead


In [19]:
diff = set(zip(df_zeros_sell['week'], df_zeros_sell['CustomerIdx'], df_zeros_sell['IsinIdx'])) - set(zip(df_ones_sell['week'], df_ones_sell['CustomerIdx'], df_ones_sell['IsinIdx']))

In [22]:
df_buy_sell = pd.DataFrame(columns=df_trade.columns)
df_buy_sell = df_buy_sell.append(df_ones_buy, ignore_index=True)
df_buy_sell = df_buy_sell.append(df_ones_sell, ignore_index=True)
df_buy_sell = df_buy_sell.append(df_zeros_buy, ignore_index=True)
df_buy_sell = df_buy_sell.append(df_zeros_sell, ignore_index=True)
df_buy_sell.sort_values(['week', 'CustomerIdx', 'IsinIdx', 'BuySell'], inplace=True)

In [32]:
df_ones_sell.head()

,CustomerIdx,IsinIdx,BuySell,CustomerInterest,month,week
867435,520,14670,Buy,0,1,1
867437,2161,6941,Buy,0,1,1
867439,2223,1915,Buy,0,1,1
867440,2249,6798,Buy,0,1,1
867441,1885,17774,Buy,0,1,1


### Merge opposite directions to original directions

In [33]:
df_ones = df_ones.append(df_ones_buy, ignore_index=True)
df_ones = df_ones.append(df_ones_sell, ignore_index=True)
df_ones.shape

(4423858, 6)

In [37]:
df_buy_sell = pd.DataFrame(columns=df_trade.columns)
df_buy_sell = df_ones.append(df_zeros, ignore_index=True)
df_buy_sell.sort_values(['week', 'CustomerIdx', 'IsinIdx', 'BuySell'], inplace=True)

In [38]:
df_buy_sell.head()

,CustomerIdx,IsinIdx,BuySell,CustomerInterest,month,week
4918670,10,312,Buy,0.0,1,0
4918672,10,316,Sell,0.0,1,0
4918704,10,2686,Sell,0.0,1,0
4918705,10,4305,Buy,0.0,1,0
4918706,10,4306,Sell,0.0,1,0


### Merge zeros and ones

### Remove duplicates

In [7]:
df_buy_sell.drop_duplicates(inplace=True)
df_buy_sell.shape

(8089037, 6)

In [ ]:
df_buy_sell.to_csv('trade_full', sep=',', index=False)